In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Proyecto de grado-Extensión/Registro de Nubes/Registro de Nubes')
!pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1bUVmxKZXlCNnu4CbA53Po31b084HWCvL/Registro de Nubes


In [2]:
import os
import tensorflow.compat.v1 as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#segundo modelo entrenado
os.chdir('/content/drive/MyDrive/PROYECTO DE GRADO/Code/DPDist-master/log/lastTrain')

## DPDist

In [4]:
def get_bn_decay(batch):
  bn_momentum = tf.train.exponential_decay(
                                            0.5,
                                            batch,#*BATCH_SIZE,
                                            float(0.5),
                                            0.5,
                                            staircase=True)

In [5]:
def initialize_uninitialized_vars(sess):
    from itertools import compress
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))

    if len(not_initialized_vars):
        print('init not_initialized')
        print(not_initialized_vars)
        sess.run(tf.variables_initializer(not_initialized_vars))

## Datos


In [6]:
def read_pc_by_class(num_point_Clouds, num_points, clase, inicial_path):
  import random
  r = random.randint(0, 20)
  lista = []
  path_i = inicial_path + '/' + clase + '/' + clase +'_'
  
  for i in range(1, num_point_Clouds+1):
    if(i<10):
      path = path_i + '00' + str(0)+str(i) + '.txt'
    if(i>=10 and i<100):
      path = path_i + '00' + str(i) + '.txt'
    if(i>=100 and i<1000):
      path = path_i + '0' + str(i) + '.txt'
    if(i>=1000):
      path = path_i + str(i) + '.txt'
    pC = np.loadtxt(path, delimiter=',', usecols=(0,1,2))
    pC = pC[r*100:r*100+num_points, :]
    lista.append(pC)



  return lista


In [7]:
def rotation_matrix(theta):
    return np.array([[np.cos(theta),-np.sin(theta), 0],
                     [np.sin(theta),np.cos(theta), 0],
                     [0,0,1]])

In [8]:
chair = read_pc_by_class(20, 5000, 'chair', '../../data/modelnet40_normal_resampled')

In [9]:
guitar = read_pc_by_class(10, 5000, 'guitar', '../../data/modelnet40_normal_resampled')

In [10]:
nube = chair[0]
SA = nube[:1024,:]
SB = nube[2048:3072,:]

In [11]:
import random

r = random.randint(1,10)
rt = random.random()
Rm = rotation_matrix(np.pi/4)
SB= np.dot(Rm,SB.T).T  

In [12]:
def get_learning_rate(batch, set_base_lr = None):
    if set_base_lr is not None:
      base_lr = set_base_lr
    else:
      base_lr = 0.00001
    learning_rate = tf.train.exponential_decay(
                        base_lr,  # Base learning rate.
                        batch, # * BATCH_SIZE,  # Current index into the dataset.
                        300*512,          # Decay step.
                        300*512,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.0000001) # CLIP THE LEARNING RATE!
    print('learning rate:')
    print(learning_rate)
    return learning_rate


In [13]:
def average_gradients(tower_grads):
  """Calculate the average gradient for each shared variable across all towers.
  Note that this function provides a synchronization point across all towers.
  From tensorflow tutorial: cifar10/cifar10_multi_gpu_train.py
  Args:
    tower_grads: List of lists of (gradient, variable) tuples. The outer list
      is over individual gradients. The inner list is over the gradient
      calculation for each tower.
  Returns:
     List of pairs of (gradient, variable) where the gradient has been averaged
     across all towers.
  """
  average_grads = []
  # print('average_gradients:')
  for grad_and_vars in zip(*tower_grads):
    # Note that each grad_and_vars looks like the following:
    #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
    grads = []
    #for g, _ in grad_and_vars:
    for g, v in grad_and_vars:
      # print(g)
      # print(v)
      # Add 0 dimension to the gradients to represent the tower.
      expanded_g = tf.expand_dims(g, 0)

      # Append on a 'tower' dimension which we will average over below.
      grads.append(expanded_g)

    # Average over the 'tower' dimension.
    grad = tf.concat(axis=0, values=grads)
    grad = tf.reduce_mean(grad, 0)

    # Keep in mind that the Variables are redundant because they are shared
    # across towers. So .. we will just return the first tower's pointer to
    # the Variable.
    v = grad_and_vars[0][1]
    grad_and_var = (grad, v)
    average_grads.append(grad_and_var)
  return average_grads

In [14]:
nube = chair[0]
SA = nube[:1024,:]
SB = nube[2048:3072,:]

In [15]:
import random

r = random.randint(1,10)
rt = random.random()
Rm = rotation_matrix(np.pi/4)
SB= np.dot(Rm,SB.T).T  + 0.5

In [16]:
import tensorflow.compat.v1 as tf
input_1 = SA.reshape(16, 64, SA.shape[1])
input_2 = SB.reshape(16,  64 ,SB.shape[1])
tower_grads_p =[]
t_grads=[]
noise = np.zeros((16, 64, 3))
from tensorflow.python.framework import meta_graph
graph = tf.Graph()
with graph.as_default():
  with tf.device('/gpu:' + str(0)):
      batch = tf.get_variable('batch', [], initializer=tf.constant_initializer(0), trainable=False)
      optimizer = tf.train.AdamOptimizer(batch,0.00001, name='MyNewAdam')
      #bn_decay = get_bn_decay(batch)
      #tf.summary.scalar('bn_decay', bn_decay) 
      is_training_pl_1 = tf.placeholder(tf.bool, shape=())
      is_training_pl_2 = tf.placeholder(tf.bool, shape=())
      x1 = tf.placeholder(tf.float32, (16, 64,3), name='input1')
      x2 = tf.placeholder(tf.float32, (16, 64,3), name='input2')
      x3 = tf.placeholder(tf.float32, (16, 64,3), name='input3')
      saver31 = tf.train.import_meta_graph('./model.ckpt.meta',
                                          import_scope='g1',
                                          input_map={'input1': x1,
                                                    'input2': x2,
                                                    'Placeholder': is_training_pl_1,
                                                    'add_noise':x3}
                                          )
      labels12 = graph.get_tensor_by_name('g1/labels12:0')
      pred_AB3 = (graph.get_tensor_by_name('g1/pc_compare/output1:0'))
      pred_BA3 = (graph.get_tensor_by_name('g1/pc_compare/output2:0'))
      loss_p = (tf.reduce_mean(pred_AB3[:,:,:,0]) + tf.reduce_mean(pred_BA3[:,:,:,0]))/2.0
      grad1 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv2/weights:0'))
      grad2 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv2/weights:0'))
      grad3 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv3/weights:0'))
      grad4 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv4/weights:0'))
      #print([n.name for n in graph.as_graph_def().node])

      train_vars_s = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='pc_compare')
      train_vars_p = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
      loss_samples = tf.get_collection('loss_samples')
      total_loss_samples = tf.add_n(loss_samples, name='total_loss_samples')
      loss_pred = tf.get_collection('loss_pred')
      total_loss_pred = tf.add_n(loss_pred, name='total_loss_pred')
      #grads_s = optimizer.compute_gradients(total_loss_samples,train_vars_s)
      grads_p = optimizer.compute_gradients(total_loss_pred,train_vars_p)
      
      tower_grads_p.append(grads_p)
      
      
      grads_p = average_gradients(tower_grads_p)
      print(grads_p)
      for g, var in grads_p:
        #print('Tensores')
        #print(g)
        #print('Variables')
        #print(var)
        t_grads.append(g)
        t_grads.append(var)
      train_op_p = optimizer.apply_gradients(grads_p, global_step=batch)
    # Create a session
  init = tf.global_variables_initializer()
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.allow_soft_placement = True
  config.log_device_placement = False
  sess3 = tf.InteractiveSession(config=config)
  sess3.run(init)
  #DPDist weights loader
  MODEL_PATH = './model.ckpt'
  saver31.restore(sess3, MODEL_PATH)
  initialize_uninitialized_vars(sess3)
  ops3 = {'x1':x1,
          'x2':x2,
          'add_noise':x3,
          # 'pc_Crec':pc_Crec,
          'is_training_pl1':is_training_pl_1,
          'loss':total_loss_pred,
          'labels12':labels12,
          'grads':grads_p,
          'train_v':train_vars_p,
          'pred': {'pred_AB':pred_AB3,'pred_BA':pred_BA3}}

  feed_dict={ops3['x1']:input_1,
            ops3['x2']: input_2,
            ops3['is_training_pl1']:False,
            ops3['add_noise']:noise,
            # train_opt_pred3:True,
            }        
  result =sess3.run([ops3['pred']], feed_dict=feed_dict)
  r = result[0]['pred_AB'].reshape(16,64,3)
  r2 = result[0]['pred_BA'].reshape(16,64,3)
 # feed_dict2={ops3['x1']:r,
         # ops3['x2']: r2,
          #ops3['is_training_pl1']:False,
        #  ops3['add_noise']:noise,
          # train_opt_pred3:True,
         # }  
  grad =sess3.run([ops3['grads']], feed_dict=feed_dict)
  t_vars = sess3.run([ops3['train_v']], feed_dict=feed_dict)
  loss = sess3.run([ops3['loss']], feed_dict=feed_dict)

[(<tf.Tensor 'Mean_2:0' shape=(1, 2503, 1, 1024) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv1/weights:0' shape=(1, 2503, 1, 1024) dtype=float32_ref>), (<tf.Tensor 'Mean_3:0' shape=(1024,) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv1/biases:0' shape=(1024,) dtype=float32_ref>), (<tf.Tensor 'Mean_4:0' shape=(1, 1, 1024, 1024) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv2/weights:0' shape=(1, 1, 1024, 1024) dtype=float32_ref>), (<tf.Tensor 'Mean_5:0' shape=(1024,) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv2/biases:0' shape=(1024,) dtype=float32_ref>), (<tf.Tensor 'Mean_6:0' shape=(1, 1, 1024, 1024) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv3/weights:0' shape=(1, 1, 1024, 1024) dtype=float32_ref>), (<tf.Tensor 'Mean_7:0' shape=(1024,) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv3/biases:0' shape=(1024,) dtype=float32_ref>), (<tf.Tensor 'Mean_

[(<tf.Tensor 'Mean_2:0' shape=(1, 2503, 1, 1024) dtype=float32>, 
<tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv1/weights:0' shape=(1, 2503, 1, 1024) dtype=float32_ref>),



(<tf.Tensor 'Mean_3:0' shape=(1024,) dtype=float32>,  
<tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv1/biases:0' shape=(1024,) dtype=float32_ref>),

(<tf.Tensor 'Mean_4:0' shape=(1, 1, 1024, 1024) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv2/weights:0' shape=(1, 1, 1024, 1024) dtype=float32_ref>),

(<tf.Tensor 'Mean_5:0' shape=(1024,) dtype=float32>, 
<tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv2/biases:0' shape=(1024,) dtype=float32_ref>),


(<tf.Tensor 'Mean_6:0' shape=(1, 1, 1024, 1024) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv3/weights:0' shape=(1, 1, 1024, 1024) dtype=float32_ref>), 


(<tf.Tensor 'Mean_7:0' shape=(1024,) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv3/biases:0' shape=(1024,) dtype=float32_ref>),


(<tf.Tensor 'Mean_8:0' shape=(1, 1, 1024, 3) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv4/weights:0' shape=(1, 1, 1024, 3) dtype=float32_ref>), 


(<tf.Tensor 'Mean_9:0' shape=(3,) dtype=float32>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv4/biases:0' shape=(3,) dtype=float32_ref>)]


In [17]:
result = np.array(result)

In [18]:
grad = np.array(grad)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [19]:
result[0]['pred_AB'].shape

(16, 64, 1, 3)

In [20]:
for i in range(8):
  print(grad[0][i,:][0].shape)
  print(grad[0][i,:][1].shape)

(1, 2503, 1, 1024)
(1, 2503, 1, 1024)
(1024,)
(1024,)
(1, 1, 1024, 1024)
(1, 1, 1024, 1024)
(1024,)
(1024,)
(1, 1, 1024, 1024)
(1, 1, 1024, 1024)
(1024,)
(1024,)
(1, 1, 1024, 3)
(1, 1, 1024, 3)
(3,)
(3,)


In [21]:
for i in range(8):
  print(grad[0][i,:][0])
  print(grad[0][i,:][1])

[[[[-3.94382323e-06  5.28635780e-07  0.00000000e+00 ...  3.72668001e-05
    -9.50062386e-05  0.00000000e+00]]

  [[-2.60988105e-04  1.05649406e-06  0.00000000e+00 ... -3.44267173e-05
     3.43754873e-05  0.00000000e+00]]

  [[ 1.39638605e-05  1.51122478e-07  0.00000000e+00 ...  1.49488158e-04
    -1.65563149e-04  0.00000000e+00]]

  ...

  [[-2.08482772e-04  5.80714001e-08  0.00000000e+00 ... -7.43248092e-05
    -3.82117287e-05  0.00000000e+00]]

  [[-2.09180405e-04 -1.05655415e-12  0.00000000e+00 ... -1.10288602e-04
    -4.50191219e-05  0.00000000e+00]]

  [[-2.27539989e-04  4.10277288e-08  0.00000000e+00 ... -6.75210176e-05
    -4.63528922e-05  0.00000000e+00]]]]
[[[[-6.2441612e-03  3.8689130e-04  7.2898460e-04 ...  2.9417434e-01
    -1.6567814e-01  2.0625185e-04]]

  [[-1.0316223e-01 -5.8178749e-04 -1.7681370e-04 ...  5.7209216e-02
    -1.3581586e-01 -1.5108447e-03]]

  [[-9.4438672e-02 -7.9198036e-04  3.8537855e-05 ...  1.4812148e-01
    -3.0481735e-01 -1.6773130e-03]]

  ...

  [[

##DPdist

In [22]:
def DPDIST(nubeA, nubeB):
  import tensorflow.compat.v1 as tf
  input_1 = nubeA.reshape(16, 64, SB.shape[1])
  input_2 = nubeB.reshape(16,  64 ,SA.shape[1])
  noise = np.zeros((16, 64, 3))
  from tensorflow.python.framework import meta_graph
  graph = tf.Graph()
  with graph.as_default():
    with tf.device('/gpu:' + str(0)):
        batch = tf.get_variable('batch', [], initializer=tf.constant_initializer(0), trainable=False)
        #bn_decay = get_bn_decay(batch)
        #tf.summary.scalar('bn_decay', bn_decay) 
        is_training_pl_1 = tf.placeholder(tf.bool, shape=())
        is_training_pl_2 = tf.placeholder(tf.bool, shape=())
        x1 = tf.placeholder(tf.float32, (16, 64,3), name='input1')
        x2 = tf.placeholder(tf.float32, (16, 64,3), name='input2')
        x3 = tf.placeholder(tf.float32, (16, 64,3), name='input3')
        saver31 = tf.train.import_meta_graph('./model.ckpt.meta',
                                            import_scope='g1',
                                            input_map={'input1': x1,
                                                      'input2': x2,
                                                      'Placeholder': is_training_pl_1,
                                                      'add_noise':x3}
                                            )
        labels12 = graph.get_tensor_by_name('g1/labels12:0')
        pred_AB3 = (graph.get_tensor_by_name('g1/pc_compare/output1:0'))
        pred_BA3 = (graph.get_tensor_by_name('g1/pc_compare/output2:0'))
        loss_p = (tf.reduce_mean(pred_AB3[:,:,:,0]) + tf.reduce_mean(pred_BA3[:,:,:,0]))/2.0
        grad1 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv2/weights:0'))
        grad2 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv2/weights:0'))
        grad3 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv3/weights:0'))
        grad4 = (graph.get_tensor_by_name('g1/pc_compare/dpdist_local/mapper_conv4/weights:0'))
        #print([n.name for n in graph.as_graph_def().node])
    # Create a session
    init = tf.global_variables_initializer()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = False
    sess3 = tf.InteractiveSession(config=config)
    sess3.run(init)
    #DPDist weights loader
    MODEL_PATH = './model.ckpt'
    saver31.restore(sess3, MODEL_PATH)
    initialize_uninitialized_vars(sess3)
    #print('model restored')
    ops3 = {'x1':x1,
            'x2':x2,
            'add_noise':x3,
            # 'pc_Crec':pc_Crec,
            'is_training_pl1':is_training_pl_1,
            'loss':loss_p,
            'labels12':labels12,
            'grads':{'conv1':grad1, 'conv2': grad2, 'conv3':grad3, 'conv4':grad4 } ,
            'pred': {'pred_AB':pred_AB3,'pred_BA':pred_BA3}}

    feed_dict={ops3['x1']:input_1,
              ops3['x2']: input_2,
              ops3['is_training_pl1']:False,
              ops3['add_noise']:noise,
              # train_opt_pred3:True,
              }        
    result =sess3.run([ops3['pred']], feed_dict=feed_dict)
    result2 = sess3.run([ops3['grads']], feed_dict=feed_dict)
    All_varaibles = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    print(All_varaibles)
    '''
    resultBA=sess3.run(pred_BA3, feed_dict=feed_dict_testing)
    resultAB_0 = resultAB[:,:,:,:]
    resultBA_0 = resultBA[:,:,:,:]  
    resultAB_0 = resultAB_0.reshape(1024,3)
    resultBA_0 = resultAB_0.reshape(1024,3)
    '''
    
    sess3.close()

    DistAB = result[0]['pred_AB']
    vDistAB = DistAB.reshape(1024,3)
    DistBA = result[0]['pred_BA']
    vDistBA = DistBA.reshape(1024,3)

    wConv1 = result2[0]['conv1']
    wConv2 = result2[0]['conv2']
    wConv3 = result2[0]['conv3']
    wConv4 = result2[0]['conv4']
    
  return result2#vDistAB, [wConv1, wConv2,wConv3, wConv4] #, vDistBA#,  #vectores de distanciaAB, #vectores de distanciaBA

In [23]:
grad = DPDIST(SA, SB)


INFO:tensorflow:Restoring parameters from ./model.ckpt


/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


[<tf.Variable 'batch:0' shape=() dtype=float32>, <tf.Variable 'g1/batch:0' shape=() dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv1/weights:0' shape=(1, 2503, 1, 1024) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv1/biases:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv2/weights:0' shape=(1, 1, 1024, 1024) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv2/biases:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv3/weights:0' shape=(1, 1, 1024, 1024) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv3/biases:0' shape=(1024,) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv4/weights:0' shape=(1, 1, 1024, 3) dtype=float32_ref>, <tf.Variable 'g1/pc_compare/dpdist_local/mapper_conv4/biases:0' shape=(3,) dtype=float32_ref>, <tf.Variable 'g1/beta1_power:0' shape=() dtype=float32_ref>, <tf.Varia

In [24]:
grad = np.array(grad)

In [26]:
for i in range(len(grad)):
  print(grad[0][:,:][0])
  print(grad[0][:,:][1])

TypeError: ignored

In [ ]:
%load_ext tensorboard

In [ ]:

os.chdir('/content/drive/MyDrive/PROYECTO DE GRADO/Code/DPDist-master/log/test1__imp_type_1enc_3dmfvEmbS512BN0LR0001wd0sigma125K5AUG1lsl1_distnoise0chairov1np64')

In [ ]:
%tensorboard --logdir __tb

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
gradients

In [ ]:
f = K.function([(SA, SB)], gradients)

In [ ]:
gradients

In [ ]:
SAT=tf.convert_to_tensor(SA)

In [ ]:
SBT=tf.convert_to_tensor(SB)

In [ ]:
def dummy_loss(x):
  return tf.reduce_sum(x)

In [ ]:
with tf.GradientTape() as t:
  outputsA, outputsB = DPDIST(SA, SB)  # This line should be within context manager
  current_loss = dummy_loss(outputsA)
  temp = t.gradient(current_loss, tf.convert_to_tensor(outputsB[1]))

In [ ]:
print(temp)

In [ ]:
with tf.GradientTape() as tape:
    tape.watch([SAT,SBT])
    result = DPDIST(SA, SB)
    max_score =tf.reduce_mean(result[0] + tf.reduce_mean(result[1]))/2.0
[dl_dw, dl_db]  = tape.gradient(max_score, [SAT,SBT])

In [ ]:
max_score

In [ ]:
dl_dw.value()

In [ ]:
input_images_tensor = tf.constant([SA, SB])
with tf.GradientTape() as g:
    g.watch((SAT, SBT))
    output_tensor = DPDIST(SA,SB)

gradients = g.gradient(tf.constant(output_tensor), input_images_tensor)

In [ ]:
print(gradients)

##Pruebas

In [ ]:
DPDIST(SA, SB)

In [ ]:
def DPDISTnormaAndDistPc_Pc(vectorDistAB, vectorDistBA):
  DPDistanceABmap = []
  for i in range(len(vectorDistAB)):
    DPDistanceAB = np.linalg.norm(vectorDistAB[i,:])
    DPDistanceABmap.append(DPDistanceAB)
  DPDistanceABmap = np.array(DPDistanceABmap)

  DPDistanceBAmap = []
  for i in range(len(vectorDistBA)):
    DPDistanceBA= np.linalg.norm(vectorDistBA[i,:])
    DPDistanceBAmap.append(DPDistanceBA)
  DPDistanceBAmap = np.array(DPDistanceBAmap)

  dist_pcA_pcB=((1/SA.shape[0])*np.sum(DPDistanceABmap))+((1/SB.shape[0])*np.sum(DPDistanceBAmap))

  return DPDistanceABmap, DPDistanceBAmap, round(dist_pcA_pcB, 4) #normaVectoresAB, #normaVectoresBA, #distancia de nube a nube 

In [ ]:
def get_bn_decay(batch):
  bn_momentum = tf.train.exponential_decay(
                                            0.5,
                                            batch,#*BATCH_SIZE,
                                            float(0.5),
                                            0.5,
                                            staircase=True)

In [ ]:
def initialize_uninitialized_vars(sess):
    from itertools import compress
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))

    if len(not_initialized_vars):
        print('init not_initialized')
        print(not_initialized_vars)
        sess.run(tf.variables_initializer(not_initialized_vars))